<h3>How to query from opeSearch<h3>

In [1]:
from sentence_transformers import SentenceTransformer
from opensearchpy import OpenSearch

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<h3>Connect to Open Search<h3>

In [2]:
#Initialize connection to opensearch
host = 'localhost'
port = 9200
auth = ('admin', 'admin') 

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False,
    timeout=100
)
#check status
print(client.info())

{'name': 'opensearch-node1_med', 'cluster_name': 'opensearch-cluster', 'cluster_uuid': 'DbPN66O6QdGNwomjb1ufRQ', 'version': {'distribution': 'opensearch', 'number': '2.11.1', 'build_type': 'tar', 'build_hash': '6b1986e964d440be9137eba1413015c31c5a7752', 'build_date': '2023-11-29T21:43:10.135035992Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.10.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'The OpenSearch Project: https://opensearch.org/'}


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\opensearchpy\connection\http_urllib3.py:208: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<h3>Knn Vector Search<h3>

In [4]:
model_name = 'intfloat/e5-base-v2'
model = SentenceTransformer(model_name)

In [5]:
query = "What is a cask disorder?"
query_vector = model.encode("query: "+query)

In [7]:
knn_search_body = {
    "size": 5,  # Number of nearest neighbors to retrieve
    "query": {
        "knn": {
            "vector": {
                "vector": query_vector,
                "k": 1  # Number of nearest neighbors to retrieve
            }
        }
    }
}

# Execute the search
response = client.search(index="med_data_strategy_3", body=knn_search_body)

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [8]:
text_result = [chunk["_source"]["text"] for chunk in response["hits"]["hits"]]
print(text_result)

['passage: clinical characteristics : cask disorders include a spectrum of phenotypes in both females and males. two main types of clinical presentation are seen : microcephalywith pontine and cerebellar hypoplasia ( micpch ), generally associated withpathogenic loss - of - function variants in cask. x - linked intellectual disability ( xlid ) with or without nystagmus, generally associated with hypomorphic caskpathogenic variants. micpch is typically seen in females with moderate - to - severeintellectual disability, progressive microcephaly with or without ophthalmologicanomalies, and sensorineural hearing loss. most are able to sit independently ; 20 % - 25 % attain the ability to walk ; language is nearly absent in most. neurologicfeatures may include axial hypotonia, hypertonia / spasticity of the extremities, and dystonia or other movement disorders. nearly 40 % have seizures by age tenyears. behaviors may include sleep disturbances, hand stereotypies, and selfbiting. micpch in m

<h3>Sparse Retrieval<h3>

In [13]:
text_query = "Second, we are committed throughout the organization to leverage expenses and improve productivity. Our goal remains ery clear: we will grow operating expenses slower than sales and grow operating income faster than sales. By lowering expenses, pssing those savings on to customers, bringing more customers in our doors, and selling more merchandise, we’re reenergizing the “productivity loop” that’s been so vital to Walmart throughout our history. We will do even more to leverage the scale, expenses and expertise of our total company all around the world. We’ll continue to make investments in technology that are clearly driving greater efficiency throughout our company. And we plan to move even quicker and be a more innovative compan"

k = 3

text_search_body = {
    "size": k, 
    "explain": True,
    "query": {
        "match": {
            "text": text_query  
        }
    }
}

response = client.search(index="wmt_financial_e5_512", body=text_search_body)


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
hits = response['hits']['hits']
for hit in hits:
    source = hit['_source']
    print(f"Score: {hit['_score']}, Text: {source['text']}")

Score: 115.09949, Text: . more than anyone in our company, our store and club associates take care of our customers and members, and they make incredible contributions to our success. im pleased with the progress that weve already made on diversity and inclusion, and especially with our global womens strategy. we are championing opportunities for women to advance in walmart, and to develop women at all levels throughout the company. driving the productivity loop delivering everyday low prices is premised on everyday low cost. there is no doubt that the pro - ductivity loop is back in walmarts dna. over the past two years, i have personally talked a lot about the productivity loop and its importance. we are very proud that we have leveraged operating expenses for two consecutive years. walmart is operating for less, so we can sell for less and drive increased sales. and we are just getting started. last october, we committed to reducing operating expenses as a percentage of sales by at 

<h3>Hybrid Search<h3>

In [ ]:
url = "/_search/pipeline/nlp_search-pipeline"
hybrid_search_pipeline ={
  "description": "Post processor for hybrid search",
  "phase_results_processors": [
    {
      "normalization-processor": {
        "normalization": {
          "technique": "min_max"
        },
        "combination": {
          "technique": "arithmetic_mean",
          "parameters": {
            "weights": [
              0.5,
              0.5
            ]
          }
        }
      }
    }
  ]
}
#client.pipeline.create("hybrid_search_pipeline", body=nlp_search_pipeline)
res = client.transport.perform_request(method = "PUT", url = url, body = hybrid_search_pipeline) 

In [ ]:
query = "Test query CASK diorder gene"
import voyageai
voyageai.api_key =  "pa-3xpcuUhVVgmOQPDBiG7ObYUA58rGn1eB1ZMaowr5xy0" 
vo = voyageai.Client()

query_embedding =  vo.embed(query, model="voyage-large-2", input_type="document").embeddings[0]

In [ ]:

route = "voyage-2-large/_search?search_pipeline=nlp_search-pipeline"
hybrid_search_body = {
  "_source": {
    "exclude": [
      "vector"
    ]
  },
  "query": {
    "hybrid": {
      "queries": [
        {
          "match": {
            "text": {
              "query": query
            }
          }
        },
        {
          "knn": {                  #instead of neural do knn search since we already have our embeddings
            "vector": {
              "vector": query_embedding,
              "k": 5
            }
          }
        }
      ]
    }
  }
}
#response = client.search(index="wmt_voyage-large-2-fixed", body=hybrid_search_body, search_type="query_then_fetch", pipeline = hybrid_search_pipeline,)
ress  = client.transport.perform_request(method = "GET", url = route, body = hybrid_search_body) 